In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
import jieba
jieba.case_sensitive = True # 可控制對於詞彙中的英文部分是否為case sensitive, 預設False
jieba.set_dictionary('dict/dict.txt.big.txt')
with open('dict/stopwords.txt', encoding='utf-8-sig') as f:
    stopwords=f.read().split('\n')

FileNotFoundError: [Errno 2] No such file or directory: 'dict/stopwords.txt'

### Import Dataset

In [ ]:
def load_data(file_path):
    file = pd.read_csv(file_path, usecols=["post_time", "title", "content"])
    return file

In [ ]:
news = load_data("bda2025_202301-202503_內容數據_新聞.csv")
dcard1 = load_data("bda2025_202301-202503_討論數據_dcard_1.csv")
dcard2 = load_data("bda2025_202301-202503_討論數據_dcard_2.csv")
mobile = load_data("bda2025_202301-202503_討論數據_mobile01.csv")
ptt = load_data("bda2025_202301-202503_討論數據_ptt.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'bda2025_202301-202503_內容數據_新聞.csv'

In [ ]:
dcard1

,post_time,title,content
0,2023-01-01 10:09:24,#分享 台灣通膨現在才注意台股當沖的人已經來不及,大家好 我是喜歡台股當沖的德國女孩 今天還看看台灣通膨和貸款利息已經變高囉 領死薪水的人未來...
1,2023-01-01 21:50:43,#分享 【投資懶人包】 | 3分鐘讓你學會如何交易股票，以及如何在股市上長久生存,股票是什麼？股票是一種金融工具，它代表著擁有一家公司的股份，可分為上市，上櫃及興櫃。當你擁有...
2,2023-01-02 00:46:16,#分享 2022-12 對帳單,https://i.imgur.com/aWD9mIa.jpg 12月 局勢由多轉空 需要到...
3,2023-01-02 01:15:41,#分享 見習新手的2022成績單,有鑑於ptt的大神們最近都在年度總結 決定也為自己結算 做一個紀錄與檢討 發dcard的原因...
4,2023-01-02 11:05:54,#新聞 中鋼 11月虧損27億元,中鋼（2002）昨（26）日公告自行結算11月合併營收320億元，單月稅前虧損27.6億元，...
...,...,...,...
14106,2023-12-31 19:00:02,#新聞 每週必追重要生活金融 #20231231,🔉［一週新聞速看］重要生活金融 ✅我們幫大家統整了上一週你必須要知道的新聞重點及摘要 想...
14107,2023-12-31 19:16:53,#其他 要怎麼操作才能有每年報籌率20%,0050年報籌率才7 8%多 美股s&p才10% 我想要長期穩定20% 要怎麼買 買哪些 怎...
14108,2023-12-31 21:34:10,#分享 2023等等要88嘍,先上一段101還沒放煙火的時候，等等再更新
14109,2023-12-31 21:41:07,#標的 12/28 華星光瀑布慘案,想了解，當天中午華星光究竟發生何事？


### 文章挑選

In [ ]:
keywords = ["房市", "台泥", "營建"]

def filter_new(file):
    df_filtered0 = file[file['content'].str.contains('|'.join(keywords), na=False)]
    df_filtered = df_filtered0[~df_filtered0['title'].str.contains("公告", na=False)]
    return df_filtered


In [ ]:
news_filtered = filter_new(news)
news_filtered

,post_time,title,content
45,2023-01-04 13:44:34,台股震盪收跌24.99點,（中央社台北4日電）台北股市今天震盪收跌，收盤跌24.99點，為14199.13點，跌幅0....
87,2023-01-05 11:08:09,台積電帶領半導體股勁揚 台股盤中漲逾百點,（中央社記者張建中台北5日電）美股收高，費城半導體指數勁揚逾2%。台股今天在台積電股價反彈，...
97,2023-01-05 19:14:24,外資大舉敲進金融股 土洋法人同步棄守電子族群,台股今 (5) 日漲逾百點，外資回頭買超 19.51 億元，台指期淨多單也增持 2406 口...
111,2023-01-06 09:07:00,打炒房時機不對！估價師：民眾將失去議價主導權,跨入2023年，全球經濟情勢尚未轉好，房市也不見走暖，卻隨即要面對持續升息、《平均地權條例》...
115,2023-01-06 10:15:39,年前賣股換現壓力大,文．高適 隨著能源價格回落、供應鏈問題改善，全球通膨觸頂已可確定，今年各國央行升息循環進入下...
...,...,...,...
558524,2025-02-28 11:23:47,《經濟》終結連四跌 消費者信心指數 2月轉升,【時報-台北電】2月份消費者信心指數（CCI）調查27日發布結果，CCI總指數72.59點，...
558525,2025-02-28 11:26:03,《傳產》打房政策壓抑、價格鬆動有限 房仲前二月交易 年減三成,【時報-台北電】打房政策壓抑、加上房價降幅有限，農曆年後房市交易量持續低迷。據各大房仲自結2...
558565,2025-02-28 15:11:19,《玻陶股》陸政策蓄勢 台玻Q2營運升溫,【時報-台北電】玻璃產業儘管現處年初傳統淡季，不過3月起逐漸有利多發酵，包括玻纖布價漲、大陸...
558592,2025-02-28 16:53:09,《熱門族群》傳產股出頭天 坐擁三大利多,【時報-台北電】美國總統川普關稅大刀狂砍，金蛇年來台股市場資金大換血，觀察2月上市類股漲幅，...


In [ ]:
dcard1_filtered = filter_new(dcard1)
dcard1_filtered

,post_time,title,content
18,2023-01-03 10:29:43,#分享 新聞及重點財經數據,俄羅斯天然氣工業公司（Gazprom）今天公布的數據顯示，2022年俄國對前蘇聯共和國組織獨...
53,2023-01-05 14:59:27,01/05 台股收盤指數,⚙️ 大盤收盤統計 ⚙️ 📌 加權指數 14301.05 / 101.92(0.72%)...
133,2023-01-11 14:59:24,01/11 台股收盤指數,⚙️ 大盤收盤統計 ⚙️ 📌 加權指數 14751.44 / -51.52(-0.35%...
161,2023-01-12 14:59:23,01/12 台股收盤指數,⚙️ 大盤收盤統計 ⚙️ 📌 加權指數 14731.64 / -19.8(-0.13%)...
165,2023-01-12 16:09:15,01/12 三大法人買賣金額統計,⚙️ 三大法人買賣金額統計 (億元) ⚙️ (單位名稱 / 買進金額 / 賣出金額 / 買...
...,...,...,...
13768,2023-12-23 20:43:24,#分享 信貸質押230萬存股投資12/22,去年2022信貸109萬持續分批投入股票市場，到現在也快告一段落了，持續分享投資心得和紀錄～...
13813,2023-12-25 14:59:32,12/25 台股收盤指數,⚙️ 大盤收盤統計 ⚙️ 📌 加權指數 17604.84 / 8.21(0.05%) 📌...
14077,2023-12-30 00:22:47,#分享 12/29 操作分享,1. 早盤億光、潤泰新突破 加碼，2. 盤末買進台泥
14094,2023-12-30 22:29:52,#請益 目前台泥都虧錢請問要繼續放嗎？,我原本買台泥是打算放長久，但是我其他股票都賺只有這個虧。建議該賣掉嗎？


In [ ]:
dcard2_filtered = filter_new(dcard2)
dcard2_filtered

,post_time,title,content
44,2024-01-02 14:59:39,01/02 台股收盤指數,⚙️ 大盤收盤統計 ⚙️ 📌 加權指數 17853.76 / -77.05(-0.43%...
46,2024-01-02 15:25:27,#分享 1/2 操作分享 又被恐懼影響 很糟糕...,1.停利應該要等到上波低點被跌破再做 還是只要 爆大量沒有繼續出量就停利？ ...
422,2024-01-09 19:11:21,#請益 新的一年打算如何配置投資組合？,台股去年最高觸碰到17000多點，身邊蠻多人在討論到底能不能站穩 個人看法是保持蠻正向的態度...
958,2024-01-22 11:50:10,#分享 1/22~/26本週美股財報列表,(以下文字內容轉自IEObserve 國際經濟觀察 ) 下週精彩財報季拉開序幕，大家最關注的...
1191,2024-01-28 20:35:27,#分享 皇昌,技術分析來看 1/26號有出量，但碰到下降趨勢線壓回明天補量高機率直接突破 停損位子會設定為...
...,...,...,...
23508,2025-02-25 21:08:48,#請益 股票這樣還有需要更換嗎？,目前都加碼0050 2884 有考慮加碼00757 其他都放著嗎？ 打算賣鴻海 台泥 62...
23553,2025-02-26 14:55:20,#請益 發行普通股數X10元後不等於股本,大家好我是股市小白，正在學習看yahoo股市的資料。 看台泥的「基本」資料時，發現它的發行普...
23566,2025-02-26 18:00:00,#標的 歷史上漲機率100％，3月高機率上漲股清單！,為什麼這些股票容易漲 受到市場上的規律或事件影響，如產業的景氣循環、公司的財報公布或是季度的...
23567,2025-02-26 18:15:27,#請益 大家是怎麼停利的？,如題 以前聽說7%～10% 停利 但我覺得科技股的話不適用 之前買的營建股終於有10%了...


In [ ]:
mobile_filtered = filter_new(mobile)
mobile_filtered

,post_time,title,content
10,2023-01-02 02:09:00,請問如果支撐cpi的售屋和租屋指標降不下來會有下一波跌勢嗎？,最近想說看一下房價和租屋出去報酬率 一個套房如果要5%報酬，買入價約120萬 而現在套房價...
19,2023-01-02 17:15:00,如果你在2008年有買房，你的結果會如何？,直接進入主題， 如果你有幸， 那個年代就進入房市， 就是我這樣的結果...
53,2023-01-06 10:07:00,【自地自建】南境日光，自在生活！開箱台南 60 坪自然極簡獨棟建築,如果條件允許，你會想要自地自建嗎？屬於南境的湛藍天空、溫軟微風，透顯著一份自在閑靜，隱身於台...
63,2023-01-07 11:16:00,現金流的重要性,現金是最好用的， 這句話應該不會有人有意見吧？ 拼了命的工作與加班， ...
91,2023-01-11 05:40:00,平均地權條例通過，全面賣出及放空房地產業相關類股！,房地產相關產業即將面臨長達2-3年甚至更長的冰凍期，相關產業股價均已呈現長空走勢。 202...
...,...,...,...
6510,2024-12-20 10:51:00,00919更換成分股,稍微研究了一下00919的成分股，研究後是不是挺有搞頭的? 11/30統計規模已經達到288...
6561,2025-01-02 11:29:00,新年第一個交易日，熱錢或回流美股？歐洲俄氣斷供，能源危機再啟！,假期期間，風險資產表現差勁，美股與幣圈落後，日元波動最大。烏克蘭切斷俄羅斯天然氣，引發能源大...
6565,2025-01-04 01:19:00,第七波打房出現成效了！,房價掉了？ 不是！ 你打火車頭，內需全死…… 台泥、亞泥，直接下看1字頭 台塑、...
6684,2025-01-24 12:53:00,每天生活費不到6.85美元（約新台幣222元）即屬貧窮,每天生活費不到6.85美元（約新台幣222元）即屬貧窮 全球億萬富豪財富 三倍速增長 經...


In [ ]:
ptt_filtered = filter_new(ptt)
ptt_filtered

,post_time,title,content
12,2023-01-01 15:06:04,[新聞] 製造業景氣加速萎縮 中國官方PMI降至47,原文標題：製造業景氣加速萎縮 中國官方PMI降至47 原文連結：https://www....
14,2023-01-01 15:33:49,[請益] 買營建股是不是穩賺不賠？,大家安安 乳題 台灣經濟火車頭護國群山房地產 投資房產賣不掉可以加價賣 去年超收的稅...
32,2023-01-02 12:36:47,Re: [新聞] 發現金或消費券不可能了？ 蔡總統提3面向,股市跌了就是股票本身跌了，一個跌 房市跌了，帶動的是代銷、仲介、建商、裝潢、清運、等百業蕭...
33,2023-01-02 12:38:46,[新聞] Q3住宅價格指數 連17漲,原文標題： Q3住宅價格指數 連17漲 原文連結： https://www.china...
38,2023-01-02 14:39:58,Re: [請益] 各種矛盾的現象,說說我的看法，希望有大大能鞭小力一點，xD Fed打通膨就是要數字好看一點，數字好看指得是...
...,...,...,...
58384,2025-02-26 16:09:29,[情報] 0226 上市外資買賣超排行,1. 標題：外資買賣超排行 2. 來源：台灣證交所 3. 網址：https://www....
58387,2025-02-26 17:00:18,[新聞] 接待中心周來人剩3~4組 代銷大佬：重回3,- 原文標題：接待中心周來人剩3~4組 代銷大佬：重回3年前盛況難 連結： http...
58417,2025-02-26 23:51:42,[新聞] 過去5年股房連動齊漲 新竹房價漲1.33倍,原文標題： 過去5年股房連動齊漲 新竹房價漲1.33倍比台股大盤更兇 原文連結： http...
58446,2025-02-27 11:25:38,"Re: [新聞] 港股收盤逼近24,000點 再超台股",等街上大媽都看得出經濟火熱的時候早不 知道漲哪裡去了 消費降級結果就是北上一線城市成長落...


### Tokenize

In [ ]:
def time_adjust(article_set):
    for index, article in article_set.iterrows():
        article["post_time"] = article["post_time"][:10]

time_adjust(news_filtered)
time_adjust(dcard1_filtered)
time_adjust(dcard2_filtered)
time_adjust(mobile_filtered)
time_adjust(ptt_filtered)

def clean_text(article):
    article_clean = re.compile(u'[^0-9a-zA-Z\u4e00-\u9fa5.]+', re.UNICODE)
    return article_clean.sub(' ', article)

def del_stopwords(tokens):
    words = []
    for token in tokens:     # 迭代產生器
        if token not in stopwords and token != ' ':     # 如果不是停用字就存入串列
            words.append(token)
    return words

def tokenization(articles):
    for index, article in articles.iterrows():
        article_clean = clean_text(article["content"])
        tokens = jieba.lcut(article_clean)
        article["content"] = del_stopwords(tokens)


In [ ]:
tokenization(news_filtered)
news_filtered

Building prefix dict from /Users/xushuting/Desktop/大三/大數據/bda2025_mid_dataset/dict/dict.txt.big.txt ...
Loading model from cache /var/folders/67/ghbhq2cn3d35t5ch2fb6wpjm0000gn/T/jieba.uc8c085db5d774315fdfc902f69ab123a.cache
Loading model cost 0.157 seconds.
Prefix dict has been built succesfully.


,post_time,title,content
45,2023-01-04,台股震盪收跌24.99點,"[中央社, 台北, 日電, 台北, 股市, 今天, 震盪, 收跌, 收盤, 跌, 24.99..."
87,2023-01-05,台積電帶領半導體股勁揚 台股盤中漲逾百點,"[中央社, 記者, 張, 建中, 台北, 日電, 美股, 收高, 費城, 半導體, 指數, ..."
97,2023-01-05,外資大舉敲進金融股 土洋法人同步棄守電子族群,"[台, 股今, 日漲, 逾, 百點, 外資, 回頭, 買超, 19.51, 億元, 台, 指..."
111,2023-01-06,打炒房時機不對！估價師：民眾將失去議價主導權,"[跨入, 2023, 年, 全球, 經濟, 情勢, 尚未, 轉好, 房市, 不見, 走暖, ..."
115,2023-01-06,年前賣股換現壓力大,"[文, 高適, 能源價格, 回落, 供應鏈, 問題, 改善, 全球通, 膨觸, 頂, 已, ..."
...,...,...,...
558524,2025-02-28,《經濟》終結連四跌 消費者信心指數 2月轉升,"[時報, 台, 北電, 月份, 消費者, 信心, 指數, CCI, 調查, 27, 日, 發..."
558525,2025-02-28,《傳產》打房政策壓抑、價格鬆動有限 房仲前二月交易 年減三成,"[時報, 台, 北電, 打房, 政策, 壓抑, 房價, 降幅, 有限, 農曆年, 後, 房市..."
558565,2025-02-28,《玻陶股》陸政策蓄勢 台玻Q2營運升溫,"[時報, 台, 北電, 玻璃, 產業, 現處, 年初, 傳統, 淡季, 月起, 逐漸, 有利..."
558592,2025-02-28,《熱門族群》傳產股出頭天 坐擁三大利多,"[時報, 台, 北電, 美國, 總統, 川普, 關稅, 大刀, 狂, 砍, 金蛇, 年來, ..."


In [ ]:
tokenization(dcard1_filtered)
dcard1_filtered

,post_time,title,content
18,2023-01-03,#分享 新聞及重點財經數據,"[俄羅斯, 天然氣, 工業, 公司, Gazprom, 今天, 公布, 數據, 顯示, 20..."
53,2023-01-05,01/05 台股收盤指數,"[大盤, 收盤, 統計, 加權指數, 14301.05, 101.92, 0.72, 櫃買,..."
133,2023-01-11,01/11 台股收盤指數,"[大盤, 收盤, 統計, 加權指數, 14751.44, 51.52, 0.35, 櫃買, ..."
161,2023-01-12,01/12 台股收盤指數,"[大盤, 收盤, 統計, 加權指數, 14731.64, 19.8, 0.13, 櫃買, 指..."
165,2023-01-12,01/12 三大法人買賣金額統計,"[法人, 買賣, 金額, 統計, 億元, 單位名稱, 買進, 金額, 賣出, 金額, 買賣,..."
...,...,...,...
13768,2023-12-23,#分享 信貸質押230萬存股投資12/22,"[去年, 2022, 信貸, 109, 萬, 持續, 分批, 投入, 股票市場, 現在, 告..."
13813,2023-12-25,12/25 台股收盤指數,"[大盤, 收盤, 統計, 加權指數, 17604.84, 8.21, 0.05, 櫃買, 指..."
14077,2023-12-30,#分享 12/29 操作分享,"[早盤, 億光, 潤泰, 新, 突破, 加碼, 盤末, 買進, 台泥]"
14094,2023-12-30,#請益 目前台泥都虧錢請問要繼續放嗎？,"[原本, 買, 台泥, 打算, 放, 長久, 股票, 賺, 虧, 建議, 賣掉]"


In [ ]:
tokenization(dcard2_filtered)
dcard2_filtered

,post_time,title,content
44,2024-01-02,01/02 台股收盤指數,"[大盤, 收盤, 統計, 加權指數, 17853.76, 77.05, 0.43, 櫃買, ..."
46,2024-01-02,#分享 1/2 操作分享 又被恐懼影響 很糟糕...,"[停利, 應該, 波, 低點, 跌破, 再, 做, 爆, 大量, 繼續, 出量, 停利, 覺..."
422,2024-01-09,#請益 新的一年打算如何配置投資組合？,"[台股, 去年, 最高, 觸, 碰到, 17000, 多點, 身邊, 人, 討論, 站穩, ..."
958,2024-01-22,#分享 1/22~/26本週美股財報列表,"[以下, 文字, 內容, 轉自, IEObserve, 國際, 經濟, 觀察, 下週, 精彩..."
1191,2024-01-28,#分享 皇昌,"[技術, 分析, 26, 號, 出量, 碰到, 下降, 趨勢, 線, 壓回, 明天, 補量,..."
...,...,...,...
23508,2025-02-25,#請益 股票這樣還有需要更換嗎？,"[目前, 加碼, 0050, 2884, 考慮, 加碼, 00757, 放, 打算, 賣鴻海..."
23553,2025-02-26,#請益 發行普通股數X10元後不等於股本,"[好, 股市, 小白, 正在, 學習, yahoo, 股市, 資料, 看台, 泥, 資料, ..."
23566,2025-02-26,#標的 歷史上漲機率100％，3月高機率上漲股清單！,"[股票, 容易, 漲, 受到, 市場, 規律, 事件, 影響, 產業, 景氣, 循環, 公司..."
23567,2025-02-26,#請益 大家是怎麼停利的？,"[題, 以前, 聽說, 10, 停利, 覺得, 科技股, 適用, 之前, 買, 營建, 股,..."


In [ ]:
tokenization(mobile_filtered)
mobile_filtered

,post_time,title,content
10,2023-01-02,請問如果支撐cpi的售屋和租屋指標降不下來會有下一波跌勢嗎？,"[最近, 想, 說, 一下, 房價, 租屋, 報酬率, 一個, 套房, 報酬, 買入價, 約..."
19,2023-01-02,如果你在2008年有買房，你的結果會如何？,"[直接, 進入, 主題, 有幸, 年代, 進入, 房市, 去年, 被動, 收入, 約, 36..."
53,2023-01-06,【自地自建】南境日光，自在生活！開箱台南 60 坪自然極簡獨棟建築,"[條件, 允許, 想要, 自地自建, 屬於, 南境, 湛藍, 天空, 溫軟, 微風, 透, ..."
63,2023-01-07,現金流的重要性,"[現金, 最好, 句話, 應該, 有人, 意見, 拼了命, 工作, 加班, 做, 身體健康,..."
91,2023-01-11,平均地權條例通過，全面賣出及放空房地產業相關類股！,"[房地產, 相關, 產業, 面臨, 長達, 年, 長, 冰凍期, 相關, 產業, 股價, 已..."
...,...,...,...
6510,2024-12-20,00919更換成分股,"[稍微, 研究, 一下, 00919, 成分股, 研究, 後, 是不是, 挺, 搞頭, 11..."
6561,2025-01-02,新年第一個交易日，熱錢或回流美股？歐洲俄氣斷供，能源危機再啟！,"[假期, 期間, 風險, 資產, 表現, 差勁, 美股, 幣圈, 落後, 日元, 波動, 最..."
6565,2025-01-04,第七波打房出現成效了！,"[房價, 掉了, 火車頭, 內需, 全死, 台泥, 亞泥, 直接, 下, 字頭, 台塑, 南..."
6684,2025-01-24,每天生活費不到6.85美元（約新台幣222元）即屬貧窮,"[每天, 生活費, 不到, 6.85, 美元, 約, 新, 台幣, 222, 元, 即屬, ..."


In [ ]:
tokenization(ptt_filtered)
ptt_filtered

,post_time,title,content
12,2023-01-01,[新聞] 製造業景氣加速萎縮 中國官方PMI降至47,"[原文, 標題, 製造業, 景氣, 加速, 萎縮, 中國, 官方, PMI, 降至, 47,..."
14,2023-01-01,[請益] 買營建股是不是穩賺不賠？,"[安安, 乳題, 台灣, 經濟, 火車頭, 護國, 群山, 房地產, 投資, 房產, 賣不掉..."
32,2023-01-02,Re: [新聞] 發現金或消費券不可能了？ 蔡總統提3面向,"[股市, 跌了, 股票, 跌了, 一個, 跌, 房市, 跌了, 帶動, 代銷, 仲介, 建商..."
33,2023-01-02,[新聞] Q3住宅價格指數 連17漲,"[原文, 標題, Q3, 住宅, 價格指數, 17, 漲, 原文, 連結, https, w..."
38,2023-01-02,Re: [請益] 各種矛盾的現象,"[說, 說, 看法, 希望, 能鞭, 小力, 一點, xD, Fed, 打通, 膨, 數字,..."
...,...,...,...
58384,2025-02-26,[情報] 0226 上市外資買賣超排行,"[標題, 外資, 買賣, 超, 排行, 來源, 台灣, 證交所, 網址, https, ww..."
58387,2025-02-26,[新聞] 接待中心周來人剩3~4組 代銷大佬：重回3,"[原文, 標題, 接待, 中心, 周來, 人, 剩, 組, 代銷, 大佬, 重回, 年前, ..."
58417,2025-02-26,[新聞] 過去5年股房連動齊漲 新竹房價漲1.33倍,"[原文, 標題, 過去, 年, 股房, 連動, 齊漲, 新竹, 房價, 漲, 1.33, 倍..."
58446,2025-02-27,"Re: [新聞] 港股收盤逼近24,000點 再超台股","[街上, 大媽, 看得出, 經濟, 火熱, 早, 知道, 漲, 裡, 消費, 降級, 北上,..."


### 漲跌標記

In [ ]:
# Import 股價漲跌幅數據
stock0 = pd.read_csv("台泥股價漲跌.csv", usecols=["日期", "漲跌價差"])
for index, day in stock0.iterrows():
    day["日期"] = datetime.strptime(day["日期"], '%Y-%m-%d')
    try:
        day["漲跌價差"] = float(day["漲跌價差"])
    except:
        pass

stock = stock0[stock0["漲跌價差"].apply(lambda x: isinstance(x, float))]

news_filtered.insert(news_filtered.shape[1], "label", 0)
dcard1_filtered.insert(dcard1_filtered.shape[1], "label", 0)
dcard2_filtered.insert(dcard2_filtered.shape[1], "label", 0)
mobile_filtered.insert(mobile_filtered.shape[1], "label", 0)
ptt_filtered.insert(ptt_filtered.shape[1], "label", 0)

In [ ]:
def label(article_set):
    for index, article in article_set.iterrows():
        day = datetime.strptime(article["post_time"], '%Y-%m-%d')
        next = 0
        for i in range(1, 20):
            try:
                next = list(stock["日期"]).index(day + timedelta(days=i))
                if next > 0:
                    break
            except:
                pass
        if stock.iloc[next]["漲跌價差"] >= 0:
            article_set["label"][index] = 1
        else:
            article_set["label"][index] = 0


In [ ]:
label(news_filtered)
news_filtered

In [ ]:
label(dcard1_filtered)
dcard1_filtered

/var/folders/67/ghbhq2cn3d35t5ch2fb6wpjm0000gn/T/ipykernel_5148/1889082649.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  article_set["label"][index] = 1
/var/folders/67/ghbhq2cn3d35t5ch2fb6wpjm0000gn/T/ipykernel_5148/1889082649.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  article_set["label"][index] = 0


,post_time,title,content,label
18,2023-01-03,#分享 新聞及重點財經數據,"[俄羅斯, 天然氣, 工業, 公司, Gazprom, 今天, 公布, 數據, 顯示, 20...",1
53,2023-01-05,01/05 台股收盤指數,"[大盤, 收盤, 統計, 加權指數, 14301.05, 101.92, 0.72, 櫃買,...",0
133,2023-01-11,01/11 台股收盤指數,"[大盤, 收盤, 統計, 加權指數, 14751.44, 51.52, 0.35, 櫃買, ...",1
161,2023-01-12,01/12 台股收盤指數,"[大盤, 收盤, 統計, 加權指數, 14731.64, 19.8, 0.13, 櫃買, 指...",1
165,2023-01-12,01/12 三大法人買賣金額統計,"[法人, 買賣, 金額, 統計, 億元, 單位名稱, 買進, 金額, 賣出, 金額, 買賣,...",1
...,...,...,...,...
13768,2023-12-23,#分享 信貸質押230萬存股投資12/22,"[去年, 2022, 信貸, 109, 萬, 持續, 分批, 投入, 股票市場, 現在, 告...",1
13813,2023-12-25,12/25 台股收盤指數,"[大盤, 收盤, 統計, 加權指數, 17604.84, 8.21, 0.05, 櫃買, 指...",1
14077,2023-12-30,#分享 12/29 操作分享,"[早盤, 億光, 潤泰, 新, 突破, 加碼, 盤末, 買進, 台泥]",0
14094,2023-12-30,#請益 目前台泥都虧錢請問要繼續放嗎？,"[原本, 買, 台泥, 打算, 放, 長久, 股票, 賺, 虧, 建議, 賣掉]",0


In [ ]:
label(dcard2_filtered)
dcard2_filtered

In [ ]:
label(mobile_filtered)
mobile_filtered

In [ ]:
label(ptt_filtered)
ptt_filtered

### Output to CSV

In [ ]:
news_filtered.to_csv("label/news_labeled.csv")
dcard1_filtered.to_csv("label/dcard1_labeled.csv")
dcard2_filtered.to_csv("label/dcard2_labeled.csv")
mobile_filtered.to_csv("label/mobile_labeled.csv")
ptt_filtered.to_csv("label/ptt_labeled.csv")

AttributeError: 'DataFrame' object has no attribute 'to_'